In [1]:
#============================================================================
# plot_fog_cases.ipynb
# Author: McKenna W. Stanford
#============================================================================

In [15]:
#==================================================
# Imports
#==================================================
import numpy as np
import matplotlib.pyplot as plt
import glob
import xarray
import datetime
import calendar
from matplotlib.gridspec import GridSpec
import matplotlib.dates as mdates
import matplotlib
import pickle
import pandas as pd
import os
from file_struct import file_struct as fs
from load_sonde_data import load_sonde_data
from give_me_files_and_subfolders import give_me_files_and_subfolders
from scipy import ndimage
from scipy.ndimage import gaussian_filter
from scipy.interpolate import NearestNDInterpolator as nn
from matplotlib.patches import Rectangle
from matplotlib import cm
import matplotlib.ticker as ticker
from scipy import stats
import matplotlib.patches as mpatches
from dask.distributed import Client, progress, LocalCluster
import time
import ctypes
#from mpl_toolkits.axes_grid.inset_locator import (inset_axes, InsetPosition,
#                                                 mark_inset)
import seaborn as sns
from matplotlib.colors import Normalize 
from scipy.interpolate import interpn
from matplotlib.lines import Line2D
import matplotlib.colors as colors
import ctypes
import matplotlib.patheffects as pe
import palettable

In [5]:
#--------------------------------------------
# Functions
#--------------------------------------------
def toTimestamp(d):
    return calendar.timegm(d.timetuple())

def toDatetime(d):
    return datetime.datetime.utcfromtimestamp(d)

def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx],idx   

# function to make serial date numbers which are the number of days that have passed
# since epoch beginning given as days.fraction_of_day
def datenum(d):
        return 366 + d.toordinal() + (d - datetime.datetime.fromordinal(d.toordinal())).total_seconds()/(24*60*60)
#--------------------------------------------
#--------------------------------------------
#--------------------------------------------
# Trim Memory
#--------------------------------------------
def trim_memory() -> int:
    libc = ctypes.CDLL("libc.so.6")
    return libc.malloc_trim(0)

In [6]:
path = '/mnt/raid/mwstanfo/micre/merged_instrument_files/'
files = glob.glob(path+'*_v3arm.p')
files = sorted(files)
print(len(files))

345


In [7]:
# make date array
dates = []
for infile in files[0:]:
    dumstr = infile.split('/')
    dumstr = dumstr[-1]
    dumstr = dumstr.split('.')
    dumstr = dumstr[0]
    dumstr = dumstr.split('_')
    dumstr = dumstr[-2]
    dumstr_year = dumstr[0:4]
    dumstr_month = dumstr[4:6]
    dumstr_day = dumstr[6:]
    date = datetime.datetime(int(dumstr_year),int(dumstr_month),int(dumstr_day))
    dates.append(date)
dates = np.array(dates)

target_date_1a = datetime.datetime(2016,4,14,0,0)
target_date_1b = datetime.datetime(2016,4,15,0,0)
target_date_2a = datetime.datetime(2016,5,21,0,0)
target_date_2b = datetime.datetime(2016,5,22,0,0)
target_date_2c = datetime.datetime(2016,5,23,0,0)
target_dates = [target_date_1a,target_date_1b,target_date_2a,target_date_2b,target_date_2c]

target_date_1_start_time = datetime.datetime(2016,4,14,23,0)
target_date_1_end_time = datetime.datetime(2016,4,16,0,0)
target_date_2_start_time = datetime.datetime(2016,5,22,11,0)
target_date_2_end_time = datetime.datetime(2016,5,23,12,0)


target_dates_ids = np.where( (dates == target_date_1a) | (dates == target_date_1b) | (dates == target_date_2a) | (dates == target_date_2b) | (dates == target_date_2c) )[0]

files_lim = []
dates_lim = []
for ii in range(len(target_dates_ids)):
    files_lim.append(files[target_dates_ids[ii]])
    dates_lim.append(dates[target_dates_ids[ii]])

In [8]:
sns.set_theme()
#sns.set_style('dark')
sns.set_style('ticks')
#sns.set(rc={'axes.facecolor':'white','axes.edgecolor': 'black','grid.color':'grey'})
sns.set_context('talk') 

In [9]:
#-----------------------------------------------------------
# Grab Variables and Merge
#-----------------------------------------------------------
merged_dict_1a = pickle.load(open(files_lim[0],"rb"))
merged_dict_1b = pickle.load(open(files_lim[1],"rb"))
merged_dict_2a = pickle.load(open(files_lim[2],"rb"))
merged_dict_2b = pickle.load(open(files_lim[3],"rb"))
merged_dict_2c = pickle.load(open(files_lim[4],"rb"))

merged_dicts = [merged_dict_1a,merged_dict_1b,merged_dict_2a,merged_dict_2b,merged_dict_2c]

basta_ref_1a = merged_dict_1a['basta']['ref']
basta_ref_1b = merged_dict_1b['basta']['ref']
basta_ref_2a = merged_dict_2a['basta']['ref']
basta_ref_2b = merged_dict_2b['basta']['ref']
basta_ref_2c = merged_dict_2c['basta']['ref']

basta_vel_1a = merged_dict_1a['basta']['vel']
basta_vel_1b = merged_dict_1b['basta']['vel']
basta_vel_2a = merged_dict_2a['basta']['vel']
basta_vel_2b = merged_dict_2b['basta']['vel']
basta_vel_2c = merged_dict_2c['basta']['vel']

basta_time_1a = merged_dict_1a['basta']['time_dt']
basta_time_1b = merged_dict_1b['basta']['time_dt']
basta_time_2a = merged_dict_2a['basta']['time_dt']
basta_time_2b = merged_dict_2b['basta']['time_dt']
basta_time_2c = merged_dict_2c['basta']['time_dt']

basta_height = merged_dict_1a['basta']['height']
height_lim_id = np.where(basta_height <= 1000.)[0]

#date1_basta_time = basta_time_1
#date1_basta_ref = basta_ref_1[height_lim_id,:]
#date1_basta_vel = basta_vel_1[height_lim_id,:]

date1_basta_time = np.concatenate((basta_time_1a,basta_time_1b))
date1_time_id = np.where( (date1_basta_time >= target_date_1_start_time) & (date1_basta_time <= target_date_1_end_time) )[0]
date1_basta_time = date1_basta_time[date1_time_id]


basta_ref_1 = np.concatenate((basta_ref_1a,basta_ref_1b),axis=1)
basta_ref_1 = basta_ref_1[:,date1_time_id]
date1_basta_ref = basta_ref_1[height_lim_id,:]

basta_vel_1 = np.concatenate((basta_vel_1a,basta_vel_1b),axis=1)
basta_vel_1 = basta_vel_1[:,date1_time_id]
date1_basta_vel = basta_vel_1[height_lim_id,:]



date2_basta_time = np.concatenate((basta_time_2a,basta_time_2b,basta_time_2c))
date2_time_id = np.where( (date2_basta_time >= target_date_2_start_time) & (date2_basta_time <= target_date_2_end_time) )[0]
date2_basta_time = date2_basta_time[date2_time_id]

basta_ref_2 = np.concatenate((basta_ref_2a,basta_ref_2b,basta_ref_2c),axis=1)
basta_ref_2 = basta_ref_2[:,date2_time_id]
date2_basta_ref = basta_ref_2[height_lim_id,:]

basta_vel_2 = np.concatenate((basta_vel_2a,basta_vel_2b,basta_vel_2c),axis=1)
basta_vel_2 = basta_vel_2[:,date2_time_id]
date2_basta_vel = basta_vel_2[height_lim_id,:]

basta_height = basta_height[height_lim_id]

In [10]:
date1a_ceil_time = merged_dict_1a['aad_ceil']['native_time_dt']
date1a_ceil_cbh = merged_dict_1a['aad_ceil']['native_cbh_1']
date1a_ceil_backscatter = merged_dict_1a['aad_ceil']['native_backscatter']
date1a_ceil_detection_status = merged_dict_1a['aad_ceil']['native_detection_status']
ceil_height = merged_dict_1a['aad_ceil']['native_range']
date1b_ceil_time = merged_dict_1b['aad_ceil']['native_time_dt']
date1b_ceil_cbh = merged_dict_1b['aad_ceil']['native_cbh_1']
date1b_ceil_backscatter = merged_dict_1b['aad_ceil']['native_backscatter']
date1b_ceil_detection_status = merged_dict_1b['aad_ceil']['native_detection_status']

date1_ceil_time = np.concatenate((date1a_ceil_time,date1b_ceil_time))
date1_ceil_cbh = np.concatenate((date1a_ceil_cbh,date1b_ceil_cbh))
date1_ceil_backscatter = np.concatenate((date1a_ceil_backscatter,date1b_ceil_backscatter))


height_lim_id = np.where(ceil_height <= 1000.)[0]
ceil_height = ceil_height[height_lim_id]

date1_time_id = np.where( (date1_ceil_time >= target_date_1_start_time) & (date1_ceil_time <= target_date_1_end_time) )[0]
date1_ceil_time = date1_ceil_time[date1_time_id]
date1_ceil_cbh = date1_ceil_cbh[date1_time_id]
date1_ceil_backscatter = date1_ceil_backscatter[date1_time_id,:]
date1_ceil_backscatter = date1_ceil_backscatter[:,height_lim_id]


date2a_ceil_time = merged_dict_2a['aad_ceil']['native_time_dt']
date2a_ceil_cbh = merged_dict_2a['aad_ceil']['native_cbh_1']
date2a_ceil_backscatter = merged_dict_2a['aad_ceil']['native_backscatter']
date2a_ceil_detection_status = merged_dict_2a['aad_ceil']['native_detection_status']

date2b_ceil_time = merged_dict_2b['aad_ceil']['native_time_dt']
date2b_ceil_cbh = merged_dict_2b['aad_ceil']['native_cbh_1']
date2b_ceil_backscatter = merged_dict_2b['aad_ceil']['native_backscatter']
date2b_ceil_detection_status = merged_dict_2b['aad_ceil']['native_detection_status']

date2c_ceil_time = merged_dict_2c['aad_ceil']['native_time_dt']
date2c_ceil_cbh = merged_dict_2c['aad_ceil']['native_cbh_1']
date2c_ceil_backscatter = merged_dict_2c['aad_ceil']['native_backscatter']
date2c_ceil_detection_status = merged_dict_2c['aad_ceil']['native_detection_status']

date2_ceil_time = np.concatenate((date2a_ceil_time,date2b_ceil_time,date2c_ceil_time))
date2_ceil_cbh = np.concatenate((date2a_ceil_cbh,date2b_ceil_cbh,date2c_ceil_cbh))
date2_ceil_backscatter = np.concatenate((date2a_ceil_backscatter,date2b_ceil_backscatter,date2c_ceil_backscatter))

date2_time_id = np.where( (date2_ceil_time >= target_date_2_start_time) & (date2_ceil_time <= target_date_2_end_time) )[0]
date2_ceil_time = date2_ceil_time[date2_time_id]
date2_ceil_cbh = date2_ceil_cbh[date2_time_id]
date2_ceil_backscatter = date2_ceil_backscatter[date2_time_id,:]

date2_ceil_backscatter = date2_ceil_backscatter[:,height_lim_id]

In [11]:
# SFC Met

# date 1
date1a_sfc_time = merged_dict_1a['sfc_met']['native_time_dt']
date1a_sfc_temp = merged_dict_1a['sfc_met']['native_temperature']
date1a_sfc_rh = merged_dict_1a['sfc_met']['native_rh']

date1b_sfc_time = merged_dict_1b['sfc_met']['native_time_dt']
date1b_sfc_temp = merged_dict_1b['sfc_met']['native_temperature']
date1b_sfc_rh = merged_dict_1b['sfc_met']['native_rh']

date1_sfc_time = np.concatenate((date1a_sfc_time,date1b_sfc_time))
date1_sfc_temp = np.concatenate((date1a_sfc_temp,date1b_sfc_temp))
date1_sfc_rh = np.concatenate((date1a_sfc_rh,date1b_sfc_rh))

date1_time_id = np.where( (date1_sfc_time >= target_date_1_start_time) & (date1_sfc_time <= target_date_1_end_time) )[0]
date1_sfc_time = list(date1_sfc_time[date1_time_id])
date1_sfc_temp = date1_sfc_temp[date1_time_id]
date1_sfc_rh = date1_sfc_rh[date1_time_id]

# date 2
date2a_sfc_time = merged_dict_2a['sfc_met']['native_time_dt']
date2a_sfc_temp = merged_dict_2a['sfc_met']['native_temperature']
date2a_sfc_rh = merged_dict_2a['sfc_met']['native_rh']

date2b_sfc_time = merged_dict_2b['sfc_met']['native_time_dt']
date2b_sfc_temp = merged_dict_2b['sfc_met']['native_temperature']
date2b_sfc_rh = merged_dict_2b['sfc_met']['native_rh']

date2c_sfc_time = merged_dict_2c['sfc_met']['native_time_dt']
date2c_sfc_temp = merged_dict_2c['sfc_met']['native_temperature']
date2c_sfc_rh = merged_dict_2c['sfc_met']['native_rh']

date2_sfc_time = np.concatenate((date2a_sfc_time,date2b_sfc_time,date2c_sfc_time))
date2_sfc_temp = np.concatenate((date2a_sfc_temp,date2b_sfc_temp,date2c_sfc_temp))
date2_sfc_rh = np.concatenate((date2a_sfc_rh,date2b_sfc_rh,date2c_sfc_rh))

date2_time_id = np.where( (date2_sfc_time >= target_date_2_start_time) & (date2_sfc_time <= target_date_2_end_time) )[0]
date2_sfc_time = list(date2_sfc_time[date2_time_id])
date2_sfc_temp = date2_sfc_temp[date2_time_id]
date2_sfc_rh = date2_sfc_rh[date2_time_id]

In [12]:
# Soundings
merged_dict_1b['native_sonde'].keys()
# Date 1
date1_sonde1_temp = merged_dict_1b['native_sonde']['1']['temperature']
date1_sonde2_temp = merged_dict_1b['native_sonde']['2']['temperature']
date1_sonde1_rh = merged_dict_1b['native_sonde']['1']['rh']
date1_sonde2_rh = merged_dict_1b['native_sonde']['2']['rh']
date1_sonde1_time = merged_dict_1b['native_sonde']['1']['time_dt']
date1_sonde2_time = merged_dict_1b['native_sonde']['2']['time_dt']
date1_sonde1_height = merged_dict_1b['native_sonde']['1']['height']
date1_sonde2_height = merged_dict_1b['native_sonde']['2']['height']
date1_sonde1_time_long = merged_dict_1b['native_sonde']['1']['time_dt_long']
date1_sonde2_time_long = merged_dict_1b['native_sonde']['2']['time_dt_long']

height_lim_id = np.where(date1_sonde1_height*1.e-3 <= 1.)[0]
date1_sonde1_temp = date1_sonde1_temp[height_lim_id]
date1_sonde1_rh = date1_sonde1_rh[height_lim_id]
date1_sonde1_height = date1_sonde1_height[height_lim_id]
date1_sonde1_time_long = np.array(date1_sonde1_time_long)[height_lim_id]

height_lim_id = np.where(date1_sonde2_height*1.e-3 <= 1.)[0]
date1_sonde2_temp = date1_sonde2_temp[height_lim_id]
date1_sonde2_rh = date1_sonde2_rh[height_lim_id]
date1_sonde2_height = date1_sonde2_height[height_lim_id]
date1_sonde2_time_long = np.array(date1_sonde2_time_long)[height_lim_id]

# Date 2
date2_sonde1_temp = merged_dict_2b['native_sonde']['2']['temperature']
date2_sonde2_temp = merged_dict_2c['native_sonde']['2']['temperature']
date2_sonde1_rh = merged_dict_2b['native_sonde']['2']['rh']
date2_sonde2_rh = merged_dict_2c['native_sonde']['2']['rh']
date2_sonde1_time = merged_dict_2b['native_sonde']['2']['time_dt']
date2_sonde2_time = merged_dict_2c['native_sonde']['2']['time_dt']
date2_sonde1_height = merged_dict_2b['native_sonde']['2']['height']
date2_sonde2_height = merged_dict_2c['native_sonde']['2']['height']
date2_sonde1_time_long = merged_dict_2b['native_sonde']['2']['time_dt_long']
date2_sonde2_time_long = merged_dict_2c['native_sonde']['2']['time_dt_long']

height_lim_id = np.where(date2_sonde1_height*1.e-3 <= 1.)[0]
date2_sonde1_temp = date2_sonde1_temp[height_lim_id]
date2_sonde1_rh = date2_sonde1_rh[height_lim_id]
date2_sonde1_height = date2_sonde1_height[height_lim_id]
date2_sonde1_time_long = np.array(date2_sonde1_time_long)[height_lim_id]

height_lim_id = np.where(date2_sonde2_height*1.e-3 <= 1.)[0]
date2_sonde2_temp = date2_sonde2_temp[height_lim_id]
date2_sonde2_rh = date2_sonde2_rh[height_lim_id]
date2_sonde2_height = date2_sonde2_height[height_lim_id]
date2_sonde2_time_long = np.array(date2_sonde2_time_long)[height_lim_id]

In [27]:
#---------------------------------------------------
# Case Study 1
#---------------------------------------------------

fig = plt.figure(figsize=(12,16))
gs = GridSpec(5,3) # 5 rows, 4 columns
Fontsize=12
dfmt = mdates.DateFormatter('%H:%M')

date1_sonde1_ax = fig.add_subplot(gs[0,0])
date1_sonde2_ax = fig.add_subplot(gs[0,1])

date1_basta_ref_ax = fig.add_subplot(gs[1,0:3])

date1_basta_vel_ax = fig.add_subplot(gs[2,0:3])

date1_ceil_ax = fig.add_subplot(gs[3,0:3])

date1_sfc_ax = fig.add_subplot(gs[4,0:3])


date1_time_axlist = [date1_basta_ref_ax,date1_basta_vel_ax,date1_ceil_ax,date1_sfc_ax]

for ax in date1_time_axlist:
    date = dates[0]
    ax.grid(which='both',ls='dotted',lw=1,c='dimgrey')
    ax.tick_params(labelsize=Fontsize)
    ax.set_xlabel('UTC Time [HH:MM]',fontsize=Fontsize)
    ax.xaxis.set_major_formatter(dfmt)
    ax.set_xlim(target_date_1_start_time,target_date_1_end_time)



sonde_ax_list = [date1_sonde1_ax,date1_sonde2_ax]
for ax in sonde_ax_list:
    ax.set_ylim(0,1)
    ax.grid(which='both',ls='dotted',lw=1,c='dimgrey')
    ax.tick_params(labelsize=Fontsize)
    ax.set_ylabel('Height [km]',fontsize=Fontsize)

    
dum_time_delta = datetime.timedelta(seconds=15)

height_axlist = [date1_basta_ref_ax,date1_basta_vel_ax,date1_ceil_ax]
for ax in height_axlist:
    ax.set_ylabel('Height [km]',fontsize=Fontsize)
    ax.set_ylim(0,1.)


#==============================================================
#==============================================================
# Date 1
#==============================================================
#==============================================================
dum_ref_ax = date1_basta_ref_ax
dum_vel_ax = date1_basta_vel_ax
dum_ceil_ax = date1_ceil_ax
dum_sfc_ax = date1_sfc_ax
dum_sonde1_ax = date1_sonde1_ax
dum_sonde2_ax = date1_sonde2_ax

#--------------------------------------------------------------
# Sounding
#--------------------------------------------------------------

# Sonde 1
dum_sonde1_ax.plot(date1_sonde1_temp-273.15,date1_sonde1_height*1.e-3,lw=2,c='red')
dum_sonde1_ax.set_xlabel('Temperature [$^{\\circ}$C]',fontsize=Fontsize,c='red')
dum_sonde1_ax.spines['bottom'].set_color('red')
dum_sonde1_ax.tick_params(axis='x',labelsize=Fontsize,colors='red')

dum_sonde1_twinax = dum_sonde1_ax.twiny()
dum_sonde1_twinax.plot(date1_sonde1_rh,date1_sonde1_height*1.e-3,lw=2,c='blue')
dum_sonde1_twinax.set_xlabel('RH [%]',fontsize=Fontsize,c='blue')
dum_sonde1_twinax.spines['top'].set_color('blue')
dum_sonde1_twinax.tick_params(axis='x',labelsize=Fontsize,colors='blue')
dum_sonde1_twinax.set_xlim(0,105)
dum_sonde1_twinax.spines['bottom'].set_visible(False)

# Find contiguous regions where rh > 95.
cloud_layer_mask = np.zeros(len(date1_sonde1_rh))
dumid = np.where(date1_sonde1_rh >= 95.)
if np.size(dumid) > 0.:
    cloud_layer_mask[dumid] = 1
cloud_Objects,num_cloud_objects = ndimage.label(cloud_layer_mask)

for kk in range(num_cloud_objects):
    dumid = np.where(cloud_Objects == (kk+1))
    dumid = np.squeeze(dumid)
    if np.size(dumid) > 1:
        date1_sonde1_ax.axhspan(date1_sonde1_height[dumid[0]]*1.e-3,date1_sonde1_height[dumid[-1]]*1.e-3,facecolor='purple',alpha=0.25)

# Sonde 2
dum_sonde2_ax.plot(date1_sonde2_temp-273.15,date1_sonde2_height*1.e-3,lw=2,c='red')
dum_sonde2_ax.set_xlabel('Temperature [$^{\\circ}$C]',fontsize=Fontsize,c='red')
dum_sonde2_ax.spines['bottom'].set_color('red')
dum_sonde2_ax.tick_params(axis='x',labelsize=Fontsize,colors='red')

dum_sonde2_twinax = dum_sonde2_ax.twiny()
dum_sonde2_twinax.plot(date1_sonde2_rh,date1_sonde2_height*1.e-3,lw=2,c='blue')
dum_sonde2_twinax.set_xlabel('RH [%]',fontsize=Fontsize,c='blue')
dum_sonde2_twinax.spines['top'].set_color('blue')
dum_sonde2_twinax.tick_params(axis='x',labelsize=Fontsize,colors='blue')
dum_sonde2_twinax.set_xlim(0,105)
dum_sonde2_twinax.spines['bottom'].set_visible(False)

# Find contiguous regions where rh > 95.
cloud_layer_mask = np.zeros(len(date1_sonde2_rh))
dumid = np.where(date1_sonde2_rh >= 95.)
if np.size(dumid) > 0.:
    cloud_layer_mask[dumid] = 1
cloud_Objects,num_cloud_objects = ndimage.label(cloud_layer_mask)

for kk in range(num_cloud_objects):
    dumid = np.where(cloud_Objects == (kk+1))
    dumid = np.squeeze(dumid)
    if np.size(dumid) > 1:
        date1_sonde2_ax.axhspan(date1_sonde2_height[dumid[0]]*1.e-3,date1_sonde2_height[dumid[-1]]*1.e-3,facecolor='purple',alpha=0.25)


#--------------------------------------------------------------
# BASTA Reflectivity
#--------------------------------------------------------------
ref = date1_basta_ref.copy()
ref[(ref > -999.) & (ref < -40.)] = -40.
ref[ref > 10.] = 10.
#cmap = matplotlib.cm.get_cmap("nipy_spectral").copy()
cmap = matplotlib.cm.get_cmap("cividis").copy()
#from palettable.cmocean.sequential import Ice_20 as colormap
#from palettable.cmocean.sequential import Solar_20 as colormap
#cmap = colormap.mpl_colormap
cmap.set_under('w')
cmap.set_bad('grey')
ref_plot = dum_ref_ax.pcolormesh(date1_basta_time,\
                          basta_height/1.e3,\
                          ref,\
                            cmap=cmap,\
                            vmin=-40.1,vmax=10.1,\
                            )      
dum_ref_ax.set_axisbelow(False)
dum_ref_ax.grid(True,c='grey',axis='both')


#--------------------------------------------------------------
# BASTA Doppler Velocity
#--------------------------------------------------------------
vel = date1_basta_vel.copy()
vel[(vel > -999.) & (vel < -2.)] = -2.
vel[vel > 2.] = 2.
cmap = matplotlib.cm.get_cmap("seismic").copy()
cmap.set_under('w')
cmap.set_bad('grey')
vel_plot = dum_vel_ax.pcolormesh(date1_basta_time,\
                          basta_height/1.e3,\
                          vel,\
                            cmap=cmap,\
                            vmin=-2.001,vmax=2.001,\
                            )      
dum_vel_ax.set_axisbelow(False)
dum_vel_ax.grid(True,c='grey',axis='both')



#--------------------------------------------------------------
# CEIL
#--------------------------------------------------------------
beta = date1_ceil_backscatter.copy()
beta[(beta > -999.) & (beta < -8.)] = -8.
beta[beta > -3.] = 3.
cmap = matplotlib.cm.get_cmap("viridis").copy()
#cmap.set_under('w')
cmap.set_bad('grey')
ceil_plot = dum_ceil_ax.pcolormesh(date1_ceil_time,\
                          ceil_height/1.e3,\
                          beta.T,\
                            cmap=cmap,\
                            vmin=-8,vmax=-3,\
                            )      
dum_ceil_ax.set_axisbelow(False)
dum_ceil_ax.grid(True,c='grey',axis='both')

# Scatterplot of CBH
dum_ceil_ax.scatter(date1_ceil_time,date1_ceil_cbh/1.e3,0.5,marker='o',c='k')
dum_ref_ax.scatter(date1_ceil_time,date1_ceil_cbh/1.e3,0.5,marker='o',c='k')
dum_vel_ax.scatter(date1_ceil_time,date1_ceil_cbh/1.e3,0.5,marker='o',c='k')


#--------------------------------------------------------------
# SFC Met
#--------------------------------------------------------------
dum_sfc_ax.plot(date1_sfc_time,date1_sfc_temp,lw=2,c='red')
dum_sfc_ax.tick_params(axis='y',labelsize=Fontsize,colors='red')
dum_sfc_ax.set_ylabel('Temperature [$^{\\circ}$C]',fontsize=Fontsize,c='red')
dum_sfc_ax.spines['left'].set_color('red')

dum_sfc_twin_ax = dum_sfc_ax.twinx()
dum_sfc_twin_ax.plot(date1_sfc_time,date1_sfc_rh,lw=2,c='blue')
dum_sfc_twin_ax.tick_params(axis='y',labelsize=Fontsize,colors='blue')
dum_sfc_twin_ax.set_ylabel('RH [%]',fontsize=Fontsize,c='blue')
dum_sfc_twin_ax.spines['right'].set_color('blue')
dum_sfc_twin_ax.spines['left'].set_visible(False)
dum_sfc_twin_ax.set_ylim(75,100.)

# Find contiguous regions where rh > 95.
cloud_layer_mask = np.zeros(len(date1_sfc_rh))
dumid = np.where(date1_sfc_rh >= 95.)
if np.size(dumid) > 0.:
    cloud_layer_mask[dumid] = 1
cloud_Objects,num_cloud_objects = ndimage.label(cloud_layer_mask)

for kk in range(num_cloud_objects):
    dumid = np.where(cloud_Objects == (kk+1))
    dumid = np.squeeze(dumid)
    if np.size(dumid) > 1:
        date1_sfc_ax.axvspan(date1_sfc_time[dumid[0]],date1_sfc_time[dumid[-1]],facecolor='purple',alpha=0.25)




#================================================
# Colorbars
#================================================

# Fixed Date 1
date1_xcbar_xstart = 0.91
date1_xcbar_xwidth = 0.015
date1_xcbar_ylength = 0.105
# Variable Date 1
date1_xcbar_ystart_ref = 0.61
date1_xcbar_ystart_vel = 0.4425
date1_xcbar_ystart_ceil = 0.275


# Date 1 Reflectivity
dum_ticks = [-40,-30,-20,-10,0,10]
ref_cbar_ax = fig.add_axes([date1_xcbar_xstart,date1_xcbar_ystart_ref,date1_xcbar_xwidth,date1_xcbar_ylength])
ref_cbar = fig.colorbar(ref_plot,orientation='vertical',cax=ref_cbar_ax,\
                           ticks=dum_ticks)
ref_cbar.ax.set_ylabel('Reflectivity [dBZ]',fontsize=Fontsize)
ref_cbar.ax.tick_params(labelsize=Fontsize)  

# Date 1 Doppler Velocity
dum_ticks = [-2.,-1.5,-1.,-0.5,0,0.5,1,1.5,2]
vel_cbar_ax = fig.add_axes([date1_xcbar_xstart,date1_xcbar_ystart_vel,date1_xcbar_xwidth,date1_xcbar_ylength])
vel_cbar = fig.colorbar(vel_plot,orientation='vertical',cax=vel_cbar_ax,\
                           ticks=dum_ticks)
vel_cbar.ax.set_ylabel('Doppler Velocity [m s$^{-1}$]',fontsize=Fontsize)
vel_cbar.ax.tick_params(labelsize=Fontsize)  

# Date 1 CEIL
dum_ticks = [-8,-7,-6,-5,-4,-3]
ceil_cbar_ax = fig.add_axes([date1_xcbar_xstart,date1_xcbar_ystart_ceil,date1_xcbar_xwidth,date1_xcbar_ylength])
ceil_cbar = fig.colorbar(ceil_plot,orientation='vertical',cax=ceil_cbar_ax,\
                           ticks=dum_ticks)
ceil_cbar.ax.set_ylabel('$log_{10}(\\beta_{att})$ [m$^{-1}$ sr$^{-1}$]',fontsize=Fontsize)
ceil_cbar.ax.tick_params(labelsize=Fontsize)  



#================================================
# Legends
#================================================
#legend_elements = [Line2D([0], [0],color='purple',ls='solid',lw=3,label='RH CTH/CBH')]
dumx = 1.4
dumy = 0.85
fac = 1.1
# RH > 95% - Sfc Met
purple_patch = mpatches.Patch(color='purple',alpha=0.25,label='RH > 95%')
lgnd1 = date1_sonde2_ax.legend(handles=[purple_patch],\
                    fontsize=Fontsize*fac,\
                    bbox_to_anchor=(dumx,dumy),\
                    ncol=1,loc='upper left',framealpha=0)

# Missing/Invalid Data
grey_patch = mpatches.Patch(color='grey',alpha=1.,label='Missing/Invalid Data')
lgnd2 = date1_sonde2_ax.legend(handles=[grey_patch],\
                    fontsize=Fontsize*fac,\
                    bbox_to_anchor=(dumx,dumy-0.2),\
                    ncol=1,loc='upper left',framealpha=0)

# CBH
legend_elements = [Line2D([0], [0],color='black',ls='solid',c='white',label='CBH',marker='o',markersize=3,markeredgecolor='k',markerfacecolor='k')]
lgnd3 = date1_sonde2_ax.legend(handles=legend_elements,\
                    fontsize=Fontsize*fac,\
                    bbox_to_anchor=(dumx,dumy-0.4),\
                    ncol=1,loc='upper left',framealpha=0)



datestr = dates_lim[0].strftime('%Y/%m/%d')

# Sonde Path
legend_elements = [Line2D([0], [0],color='brown',ls='solid',label='Sonde Path',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])]
lgnd4 = date1_sonde2_ax.legend(handles=legend_elements,\
                    fontsize=Fontsize*fac,\
                    bbox_to_anchor=(dumx,dumy-0.6),\
                    ncol=1,loc='upper left',framealpha=0)


date1_sonde2_ax.add_artist(lgnd1)
date1_sonde2_ax.add_artist(lgnd2)
date1_sonde2_ax.add_artist(lgnd3)

datestr = dates_lim[0].strftime('%Y/%m/%d')

#=======================================================
# Titles & Labels
#=======================================================
#date1_sonde1_ax.text(1.4,1.55,'15 April 2016',color='black',fontsize=Fontsize*1.5,fontweight='bold',transform=date1_sonde1_ax.transAxes,ha='center',va='center')
#date2_sonde1_ax.text(1.4,1.55,'22-23 May 2016',color='black',fontsize=Fontsize*2.25,fontweight='bold',transform=date2_sonde1_ax.transAxes,ha='center',va='center')

date1_basta_ref_ax.text(0.5,1.1,'BASTA Reflectivity',c='dimgrey',fontweight='bold',fontsize=Fontsize*1.5,transform=date1_basta_ref_ax.transAxes,ha='center',va='center')
date1_basta_vel_ax.text(0.5,1.1,'BASTA Doppler Velocity',c='dimgrey',fontweight='bold',fontsize=Fontsize*1.5,transform=date1_basta_vel_ax.transAxes,ha='center',va='center')
date1_ceil_ax.text(0.5,1.1,'Univ. of Canterbury Ceilometer $\\beta_{att}$',c='dimgrey',\
                   fontweight='bold',fontsize=Fontsize*1.5,transform=date1_ceil_ax.transAxes,ha='center',va='center')
date1_sfc_ax.text(0.5,1.1,'Surface Meteorology',c='dimgrey',\
                   fontweight='bold',fontsize=Fontsize*1.5,transform=date1_sfc_ax.transAxes,ha='center',va='center')

# Date 1 Sonde 1
dumstr=date1_sonde1_time.strftime('%H%M')
dumstr2=date1_sonde1_time.strftime('%m/%d/%Y')
dumstr='Sounding 1 @ '+dumstr+' UTC\non '+dumstr2
date1_sonde1_ax.text(0.5,1.5,dumstr,c='dimgrey',fontweight='bold',fontsize=Fontsize*1.25,transform=date1_sonde1_ax.transAxes,ha='center',va='center')

# Date 1 Sonde 2
dumstr=date1_sonde2_time.strftime('%H%M')
dumstr2=date1_sonde2_time.strftime('%m/%d/%Y')
dumstr='Sounding 2 @ '+dumstr+' UTC\non '+dumstr2
date1_sonde2_ax.text(0.5,1.5,dumstr,c='dimgrey',fontweight='bold',fontsize=Fontsize*1.25,transform=date1_sonde2_ax.transAxes,ha='center',va='center')

#======================================
# Misc. Fixes
#======================================
date1_sonde1_ax.set_xlim(5,8)
date1_sonde2_ax.set_xlim(5,8)


#======================================
# Plot Sonde Paths
#======================================

date1_basta_ref_ax.plot(date1_sonde1_time_long,date1_sonde1_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])
date1_basta_ref_ax.plot(date1_sonde2_time_long,date1_sonde2_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])

date1_basta_vel_ax.plot(date1_sonde1_time_long,date1_sonde1_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])
date1_basta_vel_ax.plot(date1_sonde2_time_long,date1_sonde2_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])

date1_ceil_ax.plot(date1_sonde1_time_long,date1_sonde1_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])
date1_ceil_ax.plot(date1_sonde2_time_long,date1_sonde2_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])

# Locate Soundings
fac = 1.
# Date 1
# Sonde1
dum = datetime.timedelta(minutes=20)
dum2 = 0.1
date1_basta_ref_ax.text(date1_sonde1_time_long[-1]+dum,(date1_sonde1_height[-1]*1.e-3)-dum2,'Sounding 1',c='brown',fontsize=Fontsize*fac,ha='left',va='center',bbox=dict(facecolor='white', alpha=1,edgecolor='deepskyblue',pad=2),fontweight='bold')
date1_basta_vel_ax.text(date1_sonde1_time_long[-1]+dum,(date1_sonde1_height[-1]*1.e-3)-dum2,'Sounding 1',c='brown',fontsize=Fontsize*fac,ha='left',va='center',bbox=dict(facecolor='white', alpha=1,edgecolor='deepskyblue',pad=2),fontweight='bold')
date1_ceil_ax.text(date1_sonde1_time_long[-1]+dum,(date1_sonde1_height[-1]*1.e-3)-dum2,'Sounding 1',c='brown',fontsize=Fontsize*fac,ha='left',va='center',bbox=dict(facecolor='white', alpha=1,edgecolor='deepskyblue',pad=2),fontweight='bold')
# Sonde2
date1_basta_ref_ax.text(date1_sonde2_time_long[-1]+dum,(date1_sonde2_height[-1]*1.e-3)-dum2,'Sounding 2',c='brown',fontsize=Fontsize*fac,ha='left',va='center',bbox=dict(facecolor='white', alpha=1,edgecolor='deepskyblue',pad=2),fontweight='bold')
date1_basta_vel_ax.text(date1_sonde2_time_long[-1]+dum,(date1_sonde2_height[-1]*1.e-3)-dum2,'Sounding 2',c='brown',fontsize=Fontsize*fac,ha='left',va='center',bbox=dict(facecolor='white', alpha=1,edgecolor='deepskyblue',pad=2),fontweight='bold')
date1_ceil_ax.text(date1_sonde2_time_long[-1]+dum,(date1_sonde2_height[-1]*1.e-3)-dum2,'Sounding 2',c='brown',fontsize=Fontsize*fac,ha='left',va='center',bbox=dict(facecolor='white', alpha=1,edgecolor='deepskyblue',pad=2),fontweight='bold')

plt.subplots_adjust(wspace=0.6,hspace=0.6)

xpos = -0.1
ypos = 1.1
fac = 2
date1_sonde1_ax.text(xpos-0.3,ypos,'(a)',fontsize=Fontsize*fac,transform=date1_sonde1_ax.transAxes)
date1_sonde2_ax.text(xpos-0.3,ypos,'(b)',fontsize=Fontsize*fac,transform=date1_sonde2_ax.transAxes)
date1_basta_ref_ax.text(xpos,ypos,'(c)',fontsize=Fontsize*fac,transform=date1_basta_ref_ax.transAxes)
date1_basta_vel_ax.text(xpos,ypos,'(d)',fontsize=Fontsize*fac,transform=date1_basta_vel_ax.transAxes)
date1_ceil_ax.text(xpos,ypos,'(e)',fontsize=Fontsize*fac,transform=date1_ceil_ax.transAxes)
date1_sfc_ax.text(xpos+0.01,ypos,'(f)',fontsize=Fontsize*fac,transform=date1_sfc_ax.transAxes)


#date1_sonde1_ax.text(1.4,1.55,'15 April 2016',color='black',fontsize=Fontsize*1.5,fontweight='bold',transform=date1_sonde1_ax.transAxes,ha='center',va='center')
plt.figtext(0.5,0.97,'15 April 2016',color='black',fontsize=Fontsize*2.5,fontweight='bold',ha='center',va='center')

fig_path = '/home/mwstanfo/figures/micre_paper/'
outfile = 'fig_e1.png'
#outfile = 'fig_b1.eps'
plt.savefig(fig_path+outfile,dpi=200,bbox_inches='tight')
#plt.show()
plt.close()
print('done')



/tmp/ipykernel_20949/3100984644.py:173: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  ceil_plot = dum_ceil_ax.pcolormesh(date1_ceil_time,\


done


In [33]:
#---------------------------------------------------
# Case Study 2
#---------------------------------------------------

fig = plt.figure(figsize=(12,16))
gs = GridSpec(5,3) # 5 rows, 4 columns
Fontsize=12
dfmt = mdates.DateFormatter('%H:%M')

date2_sonde1_ax = fig.add_subplot(gs[0,0])
date2_sonde2_ax = fig.add_subplot(gs[0,1])

date2_basta_ref_ax = fig.add_subplot(gs[1,0:3])

date2_basta_vel_ax = fig.add_subplot(gs[2,0:3])

date2_ceil_ax = fig.add_subplot(gs[3,0:3])

date2_sfc_ax = fig.add_subplot(gs[4,0:3])


date2_time_axlist = [date2_basta_ref_ax,date2_basta_vel_ax,date2_ceil_ax,date2_sfc_ax]

for ax in date2_time_axlist:
    date = dates[0]
    ax.grid(which='both',ls='dotted',lw=1,c='dimgrey')
    ax.tick_params(labelsize=Fontsize)
    ax.set_xlabel('UTC Time [HH:MM]',fontsize=Fontsize)
    ax.xaxis.set_major_formatter(dfmt)
    ax.set_xlim(target_date_2_start_time,target_date_2_end_time)



sonde_ax_list = [date2_sonde1_ax,date2_sonde2_ax]
for ax in sonde_ax_list:
    ax.set_ylim(0,1)
    ax.grid(which='both',ls='dotted',lw=1,c='dimgrey')
    ax.tick_params(labelsize=Fontsize)
    ax.set_ylabel('Height [km]',fontsize=Fontsize)

    
dum_time_delta = datetime.timedelta(seconds=15)

height_axlist = [date2_basta_ref_ax,date2_basta_vel_ax,date2_ceil_ax]
for ax in height_axlist:
    ax.set_ylabel('Height [km]',fontsize=Fontsize)
    ax.set_ylim(0,1.)


#==============================================================
#==============================================================
# Date 2
#==============================================================
#==============================================================
dum_ref_ax = date2_basta_ref_ax
dum_vel_ax = date2_basta_vel_ax
dum_ceil_ax = date2_ceil_ax
dum_sfc_ax = date2_sfc_ax
dum_sonde1_ax = date2_sonde1_ax
dum_sonde2_ax = date2_sonde2_ax

#--------------------------------------------------------------
# Sounding
#--------------------------------------------------------------

# Sonde 1
dum_sonde1_ax.plot(date2_sonde1_temp-273.15,date2_sonde1_height*1.e-3,lw=2,c='red')
dum_sonde1_ax.set_xlabel('Temperature [$^{\\circ}$C]',fontsize=Fontsize,c='red')
dum_sonde1_ax.spines['bottom'].set_color('red')
dum_sonde1_ax.tick_params(axis='x',labelsize=Fontsize,colors='red')

dum_sonde1_twinax = dum_sonde1_ax.twiny()
dum_sonde1_twinax.plot(date2_sonde1_rh,date2_sonde1_height*1.e-3,lw=2,c='blue')
dum_sonde1_twinax.set_xlabel('RH [%]',fontsize=Fontsize,c='blue')
dum_sonde1_twinax.spines['top'].set_color('blue')
dum_sonde1_twinax.tick_params(axis='x',labelsize=Fontsize,colors='blue')
dum_sonde1_twinax.set_xlim(0,105)
dum_sonde1_twinax.spines['bottom'].set_visible(False)

# Find contiguous regions where rh > 95.
cloud_layer_mask = np.zeros(len(date2_sonde1_rh))
dumid = np.where(date2_sonde1_rh >= 95.)
if np.size(dumid) > 0.:
    cloud_layer_mask[dumid] = 1
cloud_Objects,num_cloud_objects = ndimage.label(cloud_layer_mask)

for kk in range(num_cloud_objects):
    dumid = np.where(cloud_Objects == (kk+1))
    dumid = np.squeeze(dumid)
    if np.size(dumid) > 1:
        date2_sonde1_ax.axhspan(date2_sonde1_height[dumid[0]]*1.e-3,date2_sonde1_height[dumid[-1]]*1.e-3,facecolor='purple',alpha=0.25)

# Sonde 2
dum_sonde2_ax.plot(date2_sonde2_temp-273.15,date2_sonde2_height*1.e-3,lw=2,c='red')
dum_sonde2_ax.set_xlabel('Temperature [$^{\\circ}$C]',fontsize=Fontsize,c='red')
dum_sonde2_ax.spines['bottom'].set_color('red')
dum_sonde2_ax.tick_params(axis='x',labelsize=Fontsize,colors='red')

dum_sonde2_twinax = dum_sonde2_ax.twiny()
dum_sonde2_twinax.plot(date2_sonde2_rh,date2_sonde2_height*1.e-3,lw=2,c='blue')
dum_sonde2_twinax.set_xlabel('RH [%]',fontsize=Fontsize,c='blue')
dum_sonde2_twinax.spines['top'].set_color('blue')
dum_sonde2_twinax.tick_params(axis='x',labelsize=Fontsize,colors='blue')
dum_sonde2_twinax.set_xlim(0,105)
dum_sonde2_twinax.spines['bottom'].set_visible(False)

# Find contiguous regions where rh > 95.
cloud_layer_mask = np.zeros(len(date2_sonde2_rh))
dumid = np.where(date2_sonde2_rh >= 95.)
if np.size(dumid) > 0.:
    cloud_layer_mask[dumid] = 1
cloud_Objects,num_cloud_objects = ndimage.label(cloud_layer_mask)

for kk in range(num_cloud_objects):
    dumid = np.where(cloud_Objects == (kk+1))
    dumid = np.squeeze(dumid)
    if np.size(dumid) > 1:
        date2_sonde2_ax.axhspan(date2_sonde2_height[dumid[0]]*1.e-3,date2_sonde2_height[dumid[-1]]*1.e-3,facecolor='purple',alpha=0.25)


#--------------------------------------------------------------
# BASTA Reflectivity
#--------------------------------------------------------------
ref = date2_basta_ref.copy()
ref[(ref > -999.) & (ref < -40.)] = -40.
ref[ref > 10.] = 10.
cmap = matplotlib.cm.get_cmap("cividis").copy()
cmap.set_under('w')
cmap.set_bad('grey')
ref_plot = dum_ref_ax.pcolormesh(date2_basta_time,\
                          basta_height/1.e3,\
                          ref,\
                            cmap=cmap,\
                            vmin=-40.1,vmax=10.1,\
                            )      
dum_ref_ax.set_axisbelow(False)
dum_ref_ax.grid(True,c='grey',axis='both')


#--------------------------------------------------------------
# BASTA Doppler Velocity
#--------------------------------------------------------------
vel = date2_basta_vel.copy()
vel[(vel > -999.) & (vel < -2.)] = -2.
vel[vel > 2.] = 2.
cmap = matplotlib.cm.get_cmap("seismic").copy()
cmap.set_under('w')
cmap.set_bad('grey')
vel_plot = dum_vel_ax.pcolormesh(date2_basta_time,\
                          basta_height/1.e3,\
                          vel,\
                            cmap=cmap,\
                            vmin=-2.001,vmax=2.001,\
                            )      
dum_vel_ax.set_axisbelow(False)
dum_vel_ax.grid(True,c='grey',axis='both')



#--------------------------------------------------------------
# CEIL
#--------------------------------------------------------------
beta = date2_ceil_backscatter.copy()
beta[(beta > -999.) & (beta < -8.)] = -8.
beta[beta > -3.] = 3.
cmap = matplotlib.cm.get_cmap("viridis").copy()
#cmap.set_under('w')
cmap.set_bad('grey')
ceil_plot = dum_ceil_ax.pcolormesh(date2_ceil_time,\
                          ceil_height/1.e3,\
                          beta.T,\
                            cmap=cmap,\
                            vmin=-8,vmax=-3,\
                            )      
dum_ceil_ax.set_axisbelow(False)
dum_ceil_ax.grid(True,c='grey',axis='both')

# Scatterplot of CBH
dum_ceil_ax.scatter(date2_ceil_time,date2_ceil_cbh/1.e3,0.5,marker='o',c='k')
dum_ref_ax.scatter(date2_ceil_time,date2_ceil_cbh/1.e3,0.5,marker='o',c='k')
dum_vel_ax.scatter(date2_ceil_time,date2_ceil_cbh/1.e3,0.5,marker='o',c='k')


#--------------------------------------------------------------
# SFC Met
#--------------------------------------------------------------
dum_sfc_ax.plot(date2_sfc_time,date2_sfc_temp,lw=2,c='red')
dum_sfc_ax.tick_params(axis='y',labelsize=Fontsize,colors='red')
dum_sfc_ax.set_ylabel('Temperature [$^{\\circ}$C]',fontsize=Fontsize,c='red')
dum_sfc_ax.spines['left'].set_color('red')

dum_sfc_twin_ax = dum_sfc_ax.twinx()
dum_sfc_twin_ax.plot(date2_sfc_time,date2_sfc_rh,lw=2,c='blue')
dum_sfc_twin_ax.tick_params(axis='y',labelsize=Fontsize,colors='blue')
dum_sfc_twin_ax.set_ylabel('RH [%]',fontsize=Fontsize,c='blue')
dum_sfc_twin_ax.spines['right'].set_color('blue')
dum_sfc_twin_ax.spines['left'].set_visible(False)
dum_sfc_twin_ax.set_ylim(75,100.)

# Find contiguous regions where rh > 95.
cloud_layer_mask = np.zeros(len(date2_sfc_rh))
dumid = np.where(date2_sfc_rh >= 95.)
if np.size(dumid) > 0.:
    cloud_layer_mask[dumid] = 1
cloud_Objects,num_cloud_objects = ndimage.label(cloud_layer_mask)

for kk in range(num_cloud_objects):
    dumid = np.where(cloud_Objects == (kk+1))
    dumid = np.squeeze(dumid)
    if np.size(dumid) > 1:
        date2_sfc_ax.axvspan(date2_sfc_time[dumid[0]],date2_sfc_time[dumid[-1]],facecolor='purple',alpha=0.25)




#================================================
# Colorbars
#================================================

# Fixed Date 2
date2_xcbar_xstart = 0.91
date2_xcbar_xwidth = 0.015
date2_xcbar_ylength = 0.105
# Variable Date 2
date2_xcbar_ystart_ref = 0.61
date2_xcbar_ystart_vel = 0.4425
date2_xcbar_ystart_ceil = 0.275


# Date 1 Reflectivity
dum_ticks = [-50,-40,-30,-20,-10,0,10,20]
ref_cbar_ax = fig.add_axes([date2_xcbar_xstart,date2_xcbar_ystart_ref,date2_xcbar_xwidth,date2_xcbar_ylength])
ref_cbar = fig.colorbar(ref_plot,orientation='vertical',cax=ref_cbar_ax,\
                           ticks=dum_ticks)
ref_cbar.ax.set_ylabel('Reflectivity [dBZ]',fontsize=Fontsize)
ref_cbar.ax.tick_params(labelsize=Fontsize)  

# Date 1 Doppler Velocity
dum_ticks = [-2.,-1.5,-1.,-0.5,0,0.5,1,1.5,2]
vel_cbar_ax = fig.add_axes([date2_xcbar_xstart,date2_xcbar_ystart_vel,date2_xcbar_xwidth,date2_xcbar_ylength])
vel_cbar = fig.colorbar(vel_plot,orientation='vertical',cax=vel_cbar_ax,\
                           ticks=dum_ticks)
vel_cbar.ax.set_ylabel('Doppler Velocity [m s$^{-1}$]',fontsize=Fontsize)
vel_cbar.ax.tick_params(labelsize=Fontsize)  

# Date 1 CEIL
dum_ticks = [-8,-7,-6,-5,-4,-3]
ceil_cbar_ax = fig.add_axes([date2_xcbar_xstart,date2_xcbar_ystart_ceil,date2_xcbar_xwidth,date2_xcbar_ylength])
ceil_cbar = fig.colorbar(ceil_plot,orientation='vertical',cax=ceil_cbar_ax,\
                           ticks=dum_ticks)
ceil_cbar.ax.set_ylabel('$log_{10}(\\beta_{att})$ [m$^{-1}$ sr$^{-1}$]',fontsize=Fontsize)
ceil_cbar.ax.tick_params(labelsize=Fontsize)  



#================================================
# Legends
#================================================
#legend_elements = [Line2D([0], [0],color='purple',ls='solid',lw=3,label='RH CTH/CBH')]
dumx = 1.4
dumy = 0.85
fac = 1.1
# RH > 95% - Sfc Met
purple_patch = mpatches.Patch(color='purple',alpha=0.25,label='RH > 95%')
lgnd1 = date2_sonde2_ax.legend(handles=[purple_patch],\
                    fontsize=Fontsize*fac,\
                    bbox_to_anchor=(dumx,dumy),\
                    ncol=1,loc='upper left',framealpha=0)

# Missing/Invalid Data
grey_patch = mpatches.Patch(color='grey',alpha=1.,label='Missing/Invalid Data')
lgnd2 = date2_sonde2_ax.legend(handles=[grey_patch],\
                    fontsize=Fontsize*fac,\
                    bbox_to_anchor=(dumx,dumy-0.2),\
                    ncol=1,loc='upper left',framealpha=0)

# CBH
legend_elements = [Line2D([0], [0],color='black',ls='solid',c='white',label='CBH',marker='o',markersize=3,markeredgecolor='k',markerfacecolor='k')]
lgnd3 = date2_sonde2_ax.legend(handles=legend_elements,\
                    fontsize=Fontsize*fac,\
                    bbox_to_anchor=(dumx,dumy-0.4),\
                    ncol=1,loc='upper left',framealpha=0)



datestr = dates_lim[0].strftime('%Y/%m/%d')

# Sonde Path
legend_elements = [Line2D([0], [0],color='brown',ls='solid',label='Sonde Path',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])]
lgnd4 = date2_sonde2_ax.legend(handles=legend_elements,\
                    fontsize=Fontsize*fac,\
                    bbox_to_anchor=(dumx,dumy-0.6),\
                    ncol=1,loc='upper left',framealpha=0)


date2_sonde2_ax.add_artist(lgnd1)
date2_sonde2_ax.add_artist(lgnd2)
date2_sonde2_ax.add_artist(lgnd3)

datestr = dates_lim[0].strftime('%Y/%m/%d')

#=======================================================
# Titles & Labels
#=======================================================
#date2_sonde1_ax.text(1.4,1.55,'15 April 2016',color='black',fontsize=Fontsize*1.5,fontweight='bold',transform=date2_sonde1_ax.transAxes,ha='center',va='center')
#date2_sonde1_ax.text(1.4,1.55,'22-23 May 2016',color='black',fontsize=Fontsize*2.25,fontweight='bold',transform=date2_sonde1_ax.transAxes,ha='center',va='center')

date2_basta_ref_ax.text(0.5,1.1,'BASTA Reflectivity',c='dimgrey',fontweight='bold',fontsize=Fontsize*1.5,transform=date2_basta_ref_ax.transAxes,ha='center',va='center')
date2_basta_vel_ax.text(0.5,1.1,'BASTA Doppler Velocity',c='dimgrey',fontweight='bold',fontsize=Fontsize*1.5,transform=date2_basta_vel_ax.transAxes,ha='center',va='center')
date2_ceil_ax.text(0.5,1.1,'Univ. of Canterbury Ceilometer $\\beta_{att}$',c='dimgrey',\
                   fontweight='bold',fontsize=Fontsize*1.5,transform=date2_ceil_ax.transAxes,ha='center',va='center')
date2_sfc_ax.text(0.5,1.1,'Surface Meteorology',c='dimgrey',\
                   fontweight='bold',fontsize=Fontsize*1.5,transform=date2_sfc_ax.transAxes,ha='center',va='center')

# Date 1 Sonde 1
dumstr=date2_sonde1_time.strftime('%H%M')
dumstr2=date2_sonde1_time.strftime('%m/%d/%Y')
dumstr='Sounding 1 @ '+dumstr+' UTC\non '+dumstr2
date2_sonde1_ax.text(0.5,1.5,dumstr,c='dimgrey',fontweight='bold',fontsize=Fontsize*1.25,transform=date2_sonde1_ax.transAxes,ha='center',va='center')

# Date 1 Sonde 2
dumstr=date2_sonde2_time.strftime('%H%M')
dumstr2=date2_sonde2_time.strftime('%m/%d/%Y')
dumstr='Sounding 2 @ '+dumstr+' UTC\non '+dumstr2
date2_sonde2_ax.text(0.5,1.5,dumstr,c='dimgrey',fontweight='bold',fontsize=Fontsize*1.25,transform=date2_sonde2_ax.transAxes,ha='center',va='center')

#======================================
# Misc. Fixes
#======================================
date2_sonde1_ax.set_xlim(1.75,7)
date2_sonde2_ax.set_xlim(1.75,7)


#======================================
# Plot Sonde Paths
#======================================

date2_basta_ref_ax.plot(date2_sonde1_time_long,date2_sonde1_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])
date2_basta_ref_ax.plot(date2_sonde2_time_long,date2_sonde2_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])

date2_basta_vel_ax.plot(date2_sonde1_time_long,date2_sonde1_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])
date2_basta_vel_ax.plot(date2_sonde2_time_long,date2_sonde2_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])

date2_ceil_ax.plot(date2_sonde1_time_long,date2_sonde1_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])
date2_ceil_ax.plot(date2_sonde2_time_long,date2_sonde2_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])

# Locate Soundings
fac = 1.
# Date 1
# Sonde1
dum = datetime.timedelta(minutes=20)
dum2 = 0.1
date2_basta_ref_ax.text(date2_sonde1_time_long[-1]+dum,(date2_sonde1_height[-1]*1.e-3)-dum2,'Sounding 1',c='brown',fontsize=Fontsize*fac,ha='left',va='center',bbox=dict(facecolor='white', alpha=1,edgecolor='deepskyblue',pad=2),fontweight='bold')
date2_basta_vel_ax.text(date2_sonde1_time_long[-1]+dum,(date2_sonde1_height[-1]*1.e-3)-dum2,'Sounding 1',c='brown',fontsize=Fontsize*fac,ha='left',va='center',bbox=dict(facecolor='white', alpha=1,edgecolor='deepskyblue',pad=2),fontweight='bold')
date2_ceil_ax.text(date2_sonde1_time_long[-1]+dum,(date2_sonde1_height[-1]*1.e-3)-dum2,'Sounding 1',c='brown',fontsize=Fontsize*fac,ha='left',va='center',bbox=dict(facecolor='white', alpha=1,edgecolor='deepskyblue',pad=2),fontweight='bold')
# Sonde2
date2_basta_ref_ax.text(date2_sonde2_time_long[-1]-dum,(date2_sonde2_height[-1]*1.e-3)-dum2,'Sounding 2',c='brown',fontsize=Fontsize*fac,ha='right',va='center',bbox=dict(facecolor='white', alpha=1,edgecolor='deepskyblue',pad=2),fontweight='bold')
date2_basta_vel_ax.text(date2_sonde2_time_long[-1]-dum,(date2_sonde2_height[-1]*1.e-3)-dum2,'Sounding 2',c='brown',fontsize=Fontsize*fac,ha='right',va='center',bbox=dict(facecolor='white', alpha=1,edgecolor='deepskyblue',pad=2),fontweight='bold')
date2_ceil_ax.text(date2_sonde2_time_long[-1]-dum,(date2_sonde2_height[-1]*1.e-3)-dum2,'Sounding 2',c='brown',fontsize=Fontsize*fac,ha='right',va='center',bbox=dict(facecolor='white', alpha=1,edgecolor='deepskyblue',pad=2),fontweight='bold')

plt.subplots_adjust(wspace=0.6,hspace=0.6)

xpos = -0.1
ypos = 1.1
fac = 2
date2_sonde1_ax.text(xpos-0.3,ypos,'(a)',fontsize=Fontsize*fac,transform=date2_sonde1_ax.transAxes)
date2_sonde2_ax.text(xpos-0.3,ypos,'(b)',fontsize=Fontsize*fac,transform=date2_sonde2_ax.transAxes)
date2_basta_ref_ax.text(xpos,ypos,'(c)',fontsize=Fontsize*fac,transform=date2_basta_ref_ax.transAxes)
date2_basta_vel_ax.text(xpos,ypos,'(d)',fontsize=Fontsize*fac,transform=date2_basta_vel_ax.transAxes)
date2_ceil_ax.text(xpos,ypos,'(e)',fontsize=Fontsize*fac,transform=date2_ceil_ax.transAxes)
date2_sfc_ax.text(xpos+0.01,ypos,'(f)',fontsize=Fontsize*fac,transform=date2_sfc_ax.transAxes)


#date2_sonde1_ax.text(1.4,1.55,'15 April 2016',color='black',fontsize=Fontsize*1.5,fontweight='bold',transform=date2_sonde1_ax.transAxes,ha='center',va='center')
plt.figtext(0.5,0.97,'22-23 May 2016',color='black',fontsize=Fontsize*2.5,fontweight='bold',ha='center',va='center')

fig_path = '/home/mwstanfo/figures/micre_paper/'
outfile = 'fig_e2.png'
#outfile = 'fig_b1.eps'
plt.savefig(fig_path+outfile,dpi=200,bbox_inches='tight')
#plt.show()
plt.close()
print('done')



/tmp/ipykernel_20949/2336270923.py:169: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
  ceil_plot = dum_ceil_ax.pcolormesh(date2_ceil_time,\


done


In [90]:
fig = plt.figure(figsize=(36,38))
gs = GridSpec(5,4) # 5 rows, 4 columns
Fontsize=20
dfmt = mdates.DateFormatter('%H:%M')

date1_sonde1_ax = fig.add_subplot(gs[0,0])
date1_sonde2_ax = fig.add_subplot(gs[0,1])
date2_sonde1_ax = fig.add_subplot(gs[0,2])
date2_sonde2_ax = fig.add_subplot(gs[0,3])

date1_basta_ref_ax = fig.add_subplot(gs[1,0:2])
date2_basta_ref_ax = fig.add_subplot(gs[1,2:4])

date1_basta_vel_ax = fig.add_subplot(gs[2,0:2])
date2_basta_vel_ax = fig.add_subplot(gs[2,2:4])    

date1_ceil_ax = fig.add_subplot(gs[3,0:2])
date2_ceil_ax = fig.add_subplot(gs[3,2:4])

date1_sfc_ax = fig.add_subplot(gs[4,0:2])
date2_sfc_ax = fig.add_subplot(gs[4,2:4])


date1_time_axlist = [date1_basta_ref_ax,date1_basta_vel_ax,date1_ceil_ax,date1_sfc_ax]
date2_time_axlist = [date2_basta_ref_ax,date2_basta_vel_ax,date2_ceil_ax,date2_sfc_ax]

# Date 1 (Left Side)
for ax in date1_time_axlist:
    date = dates[0]
    ax.grid(which='both',ls='dotted',lw=1,c='dimgrey')
    ax.tick_params(labelsize=Fontsize)
    ax.set_xlabel('UTC Time [HH:MM]',fontsize=Fontsize)
    ax.xaxis.set_major_formatter(dfmt)
    ax.set_xlim(target_date_1_start_time,target_date_1_end_time)

# Date 2 (Right Side)
for ax in date2_time_axlist:
    datea = dates[1]
    dateb = dates[2]
    ax.grid(which='both',ls='dotted',lw=1,c='dimgrey')
    ax.tick_params(labelsize=Fontsize)
    ax.set_xlabel('UTC Time [HH:MM]',fontsize=Fontsize)
    ax.xaxis.set_major_formatter(dfmt)
    ax.set_xlim(target_date_2_start_time,target_date_2_end_time)

sonde_ax_list = [date1_sonde1_ax,date1_sonde2_ax,date2_sonde1_ax,date2_sonde2_ax]
for ax in sonde_ax_list:
    ax.set_ylim(0,1)
    ax.grid(which='both',ls='dotted',lw=1,c='dimgrey')
    ax.tick_params(labelsize=Fontsize)
    ax.set_ylabel('Height [km]',fontsize=Fontsize)

    
dum_time_delta = datetime.timedelta(seconds=15)

height_axlist = [date1_basta_ref_ax,date2_basta_ref_ax,date1_basta_vel_ax,date2_basta_vel_ax,date1_ceil_ax,date2_ceil_ax]
for ax in height_axlist:
    ax.set_ylabel('Height [km]',fontsize=Fontsize)
    ax.set_ylim(0,1.)
    
    
#==============================================================
#==============================================================
# Date 1
#==============================================================
#==============================================================
dum_ref_ax = date1_basta_ref_ax
dum_vel_ax = date1_basta_vel_ax
dum_ceil_ax = date1_ceil_ax
dum_sfc_ax = date1_sfc_ax
dum_sonde1_ax = date1_sonde1_ax
dum_sonde2_ax = date1_sonde2_ax

#--------------------------------------------------------------
# Sounding
#--------------------------------------------------------------

# Sonde 1
dum_sonde1_ax.plot(date1_sonde1_temp-273.15,date1_sonde1_height*1.e-3,lw=3,c='red')
dum_sonde1_ax.set_xlabel('Temperature [$^{\\circ}$C]',fontsize=Fontsize,c='red')
dum_sonde1_ax.spines['bottom'].set_color('red')
dum_sonde1_ax.tick_params(axis='x',labelsize=Fontsize,colors='red')

dum_sonde1_twinax = dum_sonde1_ax.twiny()
dum_sonde1_twinax.plot(date1_sonde1_rh,date1_sonde1_height*1.e-3,lw=3,c='blue')
dum_sonde1_twinax.set_xlabel('RH [%]',fontsize=Fontsize,c='blue')
dum_sonde1_twinax.spines['top'].set_color('blue')
dum_sonde1_twinax.tick_params(axis='x',labelsize=Fontsize,colors='blue')
dum_sonde1_twinax.set_xlim(0,105)
dum_sonde1_twinax.spines['bottom'].set_visible(False)

# Find contiguous regions where rh > 95.
cloud_layer_mask = np.zeros(len(date1_sonde1_rh))
dumid = np.where(date1_sonde1_rh >= 95.)
if np.size(dumid) > 0.:
    cloud_layer_mask[dumid] = 1
cloud_Objects,num_cloud_objects = ndimage.label(cloud_layer_mask)

for kk in range(num_cloud_objects):
    dumid = np.where(cloud_Objects == (kk+1))
    dumid = np.squeeze(dumid)
    if np.size(dumid) > 1:
        date1_sonde1_ax.axhspan(date1_sonde1_height[dumid[0]]*1.e-3,date1_sonde1_height[dumid[-1]]*1.e-3,facecolor='purple',alpha=0.25)

# Sonde 2
dum_sonde2_ax.plot(date1_sonde2_temp-273.15,date1_sonde2_height*1.e-3,lw=3,c='red')
dum_sonde2_ax.set_xlabel('Temperature [$^{\\circ}$C]',fontsize=Fontsize,c='red')
dum_sonde2_ax.spines['bottom'].set_color('red')
dum_sonde2_ax.tick_params(axis='x',labelsize=Fontsize,colors='red')

dum_sonde2_twinax = dum_sonde2_ax.twiny()
dum_sonde2_twinax.plot(date1_sonde2_rh,date1_sonde2_height*1.e-3,lw=3,c='blue')
dum_sonde2_twinax.set_xlabel('RH [%]',fontsize=Fontsize,c='blue')
dum_sonde2_twinax.spines['top'].set_color('blue')
dum_sonde2_twinax.tick_params(axis='x',labelsize=Fontsize,colors='blue')
dum_sonde2_twinax.set_xlim(0,105)
dum_sonde2_twinax.spines['bottom'].set_visible(False)

# Find contiguous regions where rh > 95.
cloud_layer_mask = np.zeros(len(date1_sonde2_rh))
dumid = np.where(date1_sonde2_rh >= 95.)
if np.size(dumid) > 0.:
    cloud_layer_mask[dumid] = 1
cloud_Objects,num_cloud_objects = ndimage.label(cloud_layer_mask)

for kk in range(num_cloud_objects):
    dumid = np.where(cloud_Objects == (kk+1))
    dumid = np.squeeze(dumid)
    if np.size(dumid) > 1:
        date1_sonde2_ax.axhspan(date1_sonde2_height[dumid[0]]*1.e-3,date1_sonde2_height[dumid[-1]]*1.e-3,facecolor='purple',alpha=0.25)


#--------------------------------------------------------------
# BASTA Reflectivity
#--------------------------------------------------------------
ref = date1_basta_ref.copy()
ref[(ref > -999.) & (ref < -50.)] = -50.
ref[ref > 20.] = 20.
cmap = matplotlib.cm.get_cmap("nipy_spectral").copy()
cmap.set_under('w')
cmap.set_bad('grey')
ref_plot = dum_ref_ax.pcolormesh(date1_basta_time,\
                          basta_height/1.e3,\
                          ref,\
                            cmap=cmap,\
                            vmin=-50.1,vmax=20.1,\
                            )      
dum_ref_ax.set_axisbelow(False)
dum_ref_ax.grid(True,c='grey',axis='both')


#--------------------------------------------------------------
# BASTA Doppler Velocity
#--------------------------------------------------------------
vel = date1_basta_vel.copy()
vel[(vel > -999.) & (vel < -2.)] = -2.
vel[vel > 2.] = 2.
cmap = matplotlib.cm.get_cmap("seismic").copy()
cmap.set_under('w')
cmap.set_bad('grey')
vel_plot = dum_vel_ax.pcolormesh(date1_basta_time,\
                          basta_height/1.e3,\
                          vel,\
                            cmap=cmap,\
                            vmin=-2.001,vmax=2.001,\
                            )      
dum_vel_ax.set_axisbelow(False)
dum_vel_ax.grid(True,c='grey',axis='both')



#--------------------------------------------------------------
# CEIL
#--------------------------------------------------------------
beta = date1_ceil_backscatter.copy()
beta[(beta > -999.) & (beta < -8.)] = -8.
beta[beta > -3.] = 3.
cmap = matplotlib.cm.get_cmap("jet").copy()
#cmap.set_under('w')
cmap.set_bad('grey')
ceil_plot = dum_ceil_ax.pcolormesh(date1_ceil_time,\
                          ceil_height/1.e3,\
                          beta.T,\
                            cmap=cmap,\
                            vmin=-8,vmax=-3,\
                            )      
dum_ceil_ax.set_axisbelow(False)
dum_ceil_ax.grid(True,c='grey',axis='both')

# Scatterplot of CBH
dum_ceil_ax.scatter(date1_ceil_time,date1_ceil_cbh/1.e3,5,marker='o',c='k')
dum_ref_ax.scatter(date1_ceil_time,date1_ceil_cbh/1.e3,5,marker='o',c='k')
dum_vel_ax.scatter(date1_ceil_time,date1_ceil_cbh/1.e3,5,marker='o',c='k')


#--------------------------------------------------------------
# SFC Met
#--------------------------------------------------------------
dum_sfc_ax.plot(date1_sfc_time,date1_sfc_temp,lw=3,c='red')
dum_sfc_ax.tick_params(axis='y',labelsize=Fontsize,colors='red')
dum_sfc_ax.set_ylabel('Temperature [$^{\\circ}$C]',fontsize=Fontsize,c='red')
dum_sfc_ax.spines['left'].set_color('red')

dum_sfc_twin_ax = dum_sfc_ax.twinx()
dum_sfc_twin_ax.plot(date1_sfc_time,date1_sfc_rh,lw=3,c='blue')
dum_sfc_twin_ax.tick_params(axis='y',labelsize=Fontsize,colors='blue')
dum_sfc_twin_ax.set_ylabel('RH [%]',fontsize=Fontsize,c='blue')
dum_sfc_twin_ax.spines['right'].set_color('blue')
dum_sfc_twin_ax.spines['left'].set_visible(False)
dum_sfc_twin_ax.set_ylim(75,100.)

# Find contiguous regions where rh > 95.
cloud_layer_mask = np.zeros(len(date1_sfc_rh))
dumid = np.where(date1_sfc_rh >= 95.)
if np.size(dumid) > 0.:
    cloud_layer_mask[dumid] = 1
cloud_Objects,num_cloud_objects = ndimage.label(cloud_layer_mask)

for kk in range(num_cloud_objects):
    dumid = np.where(cloud_Objects == (kk+1))
    dumid = np.squeeze(dumid)
    if np.size(dumid) > 1:
        date1_sfc_ax.axvspan(date1_sfc_time[dumid[0]],date1_sfc_time[dumid[-1]],facecolor='purple',alpha=0.25)






#==============================================================
#==============================================================
# Date 2
#==============================================================
#==============================================================
dum_ref_ax = date2_basta_ref_ax
dum_vel_ax = date2_basta_vel_ax
dum_ceil_ax = date2_ceil_ax
dum_sfc_ax = date2_sfc_ax
dum_sonde1_ax = date2_sonde1_ax
dum_sonde2_ax = date2_sonde2_ax


#--------------------------------------------------------------
# Sounding
#--------------------------------------------------------------
# Sonde 1
dum_sonde1_ax.plot(date2_sonde1_temp-273.15,date2_sonde1_height*1.e-3,lw=3,c='red')
dum_sonde1_ax.set_xlabel('Temperature [$^{\\circ}$C]',fontsize=Fontsize,c='red')
dum_sonde1_ax.spines['bottom'].set_color('red')
dum_sonde1_ax.tick_params(axis='x',labelsize=Fontsize,colors='red')

dum_sonde1_twinax = dum_sonde1_ax.twiny()
dum_sonde1_twinax.plot(date2_sonde1_rh,date2_sonde1_height*1.e-3,lw=3,c='blue')
dum_sonde1_twinax.set_xlabel('RH [%]',fontsize=Fontsize,c='blue')
dum_sonde1_twinax.spines['top'].set_color('blue')
dum_sonde1_twinax.tick_params(axis='x',labelsize=Fontsize,colors='blue')
dum_sonde1_twinax.set_xlim(0,105)
dum_sonde1_twinax.spines['bottom'].set_visible(False)

# Find contiguous regions where rh > 95.
cloud_layer_mask = np.zeros(len(date2_sonde1_rh))
dumid = np.where(date2_sonde1_rh >= 95.)
if np.size(dumid) > 0.:
    cloud_layer_mask[dumid] = 1
cloud_Objects,num_cloud_objects = ndimage.label(cloud_layer_mask)

for kk in range(num_cloud_objects):
    dumid = np.where(cloud_Objects == (kk+1))
    dumid = np.squeeze(dumid)
    if np.size(dumid) > 1:
        date2_sonde1_ax.axhspan(date2_sonde1_height[dumid[0]]*1.e-3,date2_sonde1_height[dumid[-1]]*1.e-3,facecolor='purple',alpha=0.25)

# Sonde 2
dum_sonde2_ax.plot(date2_sonde2_temp-273.15,date2_sonde2_height*1.e-3,lw=3,c='red')
dum_sonde2_ax.set_xlabel('Temperature [$^{\\circ}$C]',fontsize=Fontsize,c='red')
dum_sonde2_ax.spines['bottom'].set_color('red')
dum_sonde2_ax.tick_params(axis='x',labelsize=Fontsize,colors='red')

dum_sonde2_twinax = dum_sonde2_ax.twiny()
dum_sonde2_twinax.plot(date2_sonde2_rh,date2_sonde2_height*1.e-3,lw=3,c='blue')
dum_sonde2_twinax.set_xlabel('RH [%]',fontsize=Fontsize,c='blue')
dum_sonde2_twinax.spines['top'].set_color('blue')
dum_sonde2_twinax.tick_params(axis='x',labelsize=Fontsize,colors='blue')
dum_sonde2_twinax.set_xlim(0,105)
dum_sonde2_twinax.spines['bottom'].set_visible(False)

# Find contiguous regions where rh > 95.
cloud_layer_mask = np.zeros(len(date2_sonde2_rh))
dumid = np.where(date2_sonde2_rh >= 95.)
if np.size(dumid) > 0.:
    cloud_layer_mask[dumid] = 1
cloud_Objects,num_cloud_objects = ndimage.label(cloud_layer_mask)

for kk in range(num_cloud_objects):
    dumid = np.where(cloud_Objects == (kk+1))
    dumid = np.squeeze(dumid)
    if np.size(dumid) > 1:
        date2_sonde2_ax.axhspan(date2_sonde2_height[dumid[0]]*1.e-3,date2_sonde2_height[dumid[-1]]*1.e-3,facecolor='purple',alpha=0.25)


#--------------------------------------------------------------
# BASTA Reflectivity
#--------------------------------------------------------------
ref = date2_basta_ref.copy()
ref[(ref > -999.) & (ref < -50.)] = -50.
ref[ref > 20.] = 20.
cmap = matplotlib.cm.get_cmap("nipy_spectral").copy()
cmap.set_under('w')
cmap.set_bad('grey')
ref_plot = dum_ref_ax.pcolormesh(date2_basta_time,\
                          basta_height/1.e3,\
                          ref,\
                            cmap=cmap,\
                            vmin=-50.1,vmax=20.1,\
                            )      
dum_ref_ax.set_axisbelow(False)
dum_ref_ax.grid(True,c='grey',axis='both')
#--------------------------------------------------------------
# BASTA Doppler Velocity
#--------------------------------------------------------------
vel = date2_basta_vel.copy()
vel[(vel > -999.) & (vel < -2.)] = -2.
vel[vel > 2.] = 2.
cmap = matplotlib.cm.get_cmap("seismic").copy()
cmap.set_under('w')
cmap.set_bad('grey')
vel_plot = dum_vel_ax.pcolormesh(date2_basta_time,\
                          basta_height/1.e3,\
                          vel,\
                            cmap=cmap,\
                            vmin=-2.001,vmax=2.001,\
                            )      
dum_vel_ax.set_axisbelow(False)
dum_vel_ax.grid(True,c='grey',axis='both')

#--------------------------------------------------------------
# CEIL
#--------------------------------------------------------------
beta = date2_ceil_backscatter.copy()
beta[(beta > -999.) & (beta < -8.)] = -8.
beta[beta > -3.] = 3.
cmap = matplotlib.cm.get_cmap("jet").copy()
#cmap.set_under('w')
cmap.set_bad('grey')
ceil_plot = dum_ceil_ax.pcolormesh(date2_ceil_time,\
                          ceil_height/1.e3,\
                          beta.T,\
                            cmap=cmap,\
                            vmin=-8,vmax=-3,\
                            )      
dum_ceil_ax.set_axisbelow(False)
dum_ceil_ax.grid(True,c='grey',axis='both')

# Scatterplot of CBH
dum_ceil_ax.scatter(date2_ceil_time,date2_ceil_cbh/1.e3,5,marker='o',c='k')
dum_ref_ax.scatter(date2_ceil_time,date2_ceil_cbh/1.e3,5,marker='o',c='k')
dum_vel_ax.scatter(date2_ceil_time,date2_ceil_cbh/1.e3,5,marker='o',c='k')

#--------------------------------------------------------------
# SFC Met
#--------------------------------------------------------------
dum_sfc_ax.plot(date2_sfc_time,date2_sfc_temp,lw=3,c='red')
dum_sfc_ax.tick_params(axis='y',labelsize=Fontsize,colors='red')
dum_sfc_ax.set_ylabel('Temperature [$^{\\circ}$C]',fontsize=Fontsize,c='red')
dum_sfc_ax.spines['left'].set_color('red')
dum_sfc_twin_ax.spines['right'].set_visible(False)


dum_sfc_twin_ax = dum_sfc_ax.twinx()
dum_sfc_twin_ax.plot(date2_sfc_time,date2_sfc_rh,lw=3,c='blue')
dum_sfc_twin_ax.tick_params(axis='y',labelsize=Fontsize,colors='blue')
dum_sfc_twin_ax.set_ylabel('RH [%]',fontsize=Fontsize,c='blue')
dum_sfc_twin_ax.spines['right'].set_color('blue')
dum_sfc_twin_ax.spines['left'].set_visible(False)
dum_sfc_twin_ax.set_ylim(75,100.)

# Find contiguous regions where rh > 95.
cloud_layer_mask = np.zeros(len(date2_sfc_rh))
dumid = np.where(date2_sfc_rh >= 95.)
if np.size(dumid) > 0.:
    cloud_layer_mask[dumid] = 1
cloud_Objects,num_cloud_objects = ndimage.label(cloud_layer_mask)

for kk in range(num_cloud_objects):
    dumid = np.where(cloud_Objects == (kk+1))
    dumid = np.squeeze(dumid)
    if np.size(dumid) > 1:
        date2_sfc_ax.axvspan(date2_sfc_time[dumid[0]],date2_sfc_time[dumid[-1]],facecolor='purple',alpha=0.25)


#================================================
# Colorbars
#================================================

# Fixed Date 1
date1_xcbar_xstart = 0.48
date1_xcbar_xwidth = 0.01
date1_xcbar_ylength = 0.11
# Variable Date 1
date1_xcbar_ystart_ref = 0.61
date1_xcbar_ystart_vel = 0.4475
date1_xcbar_ystart_ceil = 0.285

# Fixed Date 2
date2_xcbar_xstart = 0.91
date2_xcbar_xwidth = date1_xcbar_xwidth
date2_xcbar_ylength = date1_xcbar_ylength
# Variable Date 2
date2_xcbar_ystart_ref = date1_xcbar_ystart_ref
date2_xcbar_ystart_vel = date1_xcbar_ystart_vel
date2_xcbar_ystart_ceil = date1_xcbar_ystart_ceil


# Date 1 Reflectivity
dum_ticks = [-50,-40,-30,-20,-10,0,10,20]
ref_cbar_ax = fig.add_axes([date1_xcbar_xstart,date1_xcbar_ystart_ref,date1_xcbar_xwidth,date1_xcbar_ylength])
ref_cbar = fig.colorbar(ref_plot,orientation='vertical',cax=ref_cbar_ax,\
                           ticks=dum_ticks)
ref_cbar.ax.set_ylabel('Reflectivity [dBZ]',fontsize=Fontsize)
ref_cbar.ax.tick_params(labelsize=Fontsize)  

# Date 1 Doppler Velocity
dum_ticks = [-2.,-1.5,-1.,-0.5,0,0.5,1,1.5,2]
vel_cbar_ax = fig.add_axes([date1_xcbar_xstart,date1_xcbar_ystart_vel,date1_xcbar_xwidth,date1_xcbar_ylength])
vel_cbar = fig.colorbar(vel_plot,orientation='vertical',cax=vel_cbar_ax,\
                           ticks=dum_ticks)
vel_cbar.ax.set_ylabel('Doppler Velocity [m s$^{-1}$]',fontsize=Fontsize)
vel_cbar.ax.tick_params(labelsize=Fontsize)  

# Date 1 CEIL
dum_ticks = [-8,-7,-6,-5,-4,-3]
ceil_cbar_ax = fig.add_axes([date1_xcbar_xstart,date1_xcbar_ystart_ceil,date1_xcbar_xwidth,date1_xcbar_ylength])
ceil_cbar = fig.colorbar(ceil_plot,orientation='vertical',cax=ceil_cbar_ax,\
                           ticks=dum_ticks)
ceil_cbar.ax.set_ylabel('$log_{10}(\\beta_{att})$ [m$^{-1}$ sr$^{-1}$]',fontsize=Fontsize)
ceil_cbar.ax.tick_params(labelsize=Fontsize)  


# Date 2 Reflectivity
dum_ticks = [-50,-40,-30,-20,-10,0,10,20]
ref_cbar_ax = fig.add_axes([date2_xcbar_xstart,date2_xcbar_ystart_ref,date2_xcbar_xwidth,date2_xcbar_ylength])
ref_cbar = fig.colorbar(ref_plot,orientation='vertical',cax=ref_cbar_ax,\
                           ticks=dum_ticks)
ref_cbar.ax.set_ylabel('Reflectivity [dBZ]',fontsize=Fontsize)
ref_cbar.ax.tick_params(labelsize=Fontsize)  

# Date 2 Doppler Velocity
dum_ticks = [-2.,-1.5,-1.,-0.5,0,0.5,1,1.5,2]
vel_cbar_ax = fig.add_axes([date2_xcbar_xstart,date2_xcbar_ystart_vel,date2_xcbar_xwidth,date2_xcbar_ylength])
vel_cbar = fig.colorbar(vel_plot,orientation='vertical',cax=vel_cbar_ax,\
                           ticks=dum_ticks)
vel_cbar.ax.set_ylabel('Doppler Velocity [m s$^{-1}$]',fontsize=Fontsize)
vel_cbar.ax.tick_params(labelsize=Fontsize)  

# Date 2 CEIL
dum_ticks = [-8,-7,-6,-5,-4,-3]
ceil_cbar_ax = fig.add_axes([date2_xcbar_xstart,date2_xcbar_ystart_ceil,date2_xcbar_xwidth,date2_xcbar_ylength])
ceil_cbar = fig.colorbar(ceil_plot,orientation='vertical',cax=ceil_cbar_ax,\
                           ticks=dum_ticks)
ceil_cbar.ax.set_ylabel('$log_{10}(\\beta_{att})$ [m$^{-1}$ sr$^{-1}$]',fontsize=Fontsize)
ceil_cbar.ax.tick_params(labelsize=Fontsize)  


#================================================
# Legends
#================================================
#legend_elements = [Line2D([0], [0],color='purple',ls='solid',lw=3,label='RH CTH/CBH')]

# RH > 95% - Sfc Met
purple_patch = mpatches.Patch(color='purple',alpha=0.25,label='RH > 95%')
lgnd1 = date1_sfc_ax.legend(handles=[purple_patch],\
                    fontsize=Fontsize*1.5,\
                    bbox_to_anchor=(1.3,1.3),\
                    ncol=1,loc='upper right',framealpha=0)

# RH > 95% - Sonde
purple_patch = mpatches.Patch(color='purple',alpha=0.25,label='RH > 95%')
lgnd2 = date1_sonde2_ax.legend(handles=[purple_patch],\
                    fontsize=Fontsize*1.5,\
                    bbox_to_anchor=(1.75,1.35),\
                    ncol=1,loc='upper right',framealpha=0)

# Missing/Invalid Data
grey_patch = mpatches.Patch(color='grey',alpha=1.,label='Missing/Invalid Data')
lgnd3 = date1_basta_ref_ax.legend(handles=[grey_patch],\
                    fontsize=Fontsize*1.5,\
                    bbox_to_anchor=(1.35,1.275),\
                    ncol=1,loc='upper right',framealpha=0)

# CBH
legend_elements = [Line2D([0], [0],color='black',ls='solid',c='white',label='CBH',marker='o',markersize=10,markeredgecolor='k',markerfacecolor='k')]
lgnd4 = date1_basta_vel_ax.legend(handles=legend_elements,\
                    fontsize=Fontsize*1.5,\
                    bbox_to_anchor=(1.2,1.4),\
                    ncol=1,loc='upper right',framealpha=0)



datestr = dates_lim[0].strftime('%Y/%m/%d')

# Sonde Path
legend_elements = [Line2D([0], [0],color='brown',ls='solid',label='Sonde Path',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])]
lgnd5 = date1_basta_vel_ax.legend(handles=legend_elements,\
                    fontsize=Fontsize*1.5,\
                    bbox_to_anchor=(1.3,1.275),\
                    ncol=1,loc='upper right',framealpha=0)

date1_basta_vel_ax.add_artist(lgnd4)

datestr = dates_lim[0].strftime('%Y/%m/%d')

#=======================================================
# Titles & Labels
#=======================================================
date1_sonde1_ax.text(1.4,1.55,'15 April 2016',color='black',fontsize=Fontsize*2.25,fontweight='bold',transform=date1_sonde1_ax.transAxes,ha='center',va='center')
date2_sonde1_ax.text(1.4,1.55,'22-23 May 2016',color='black',fontsize=Fontsize*2.25,fontweight='bold',transform=date2_sonde1_ax.transAxes,ha='center',va='center')

date1_basta_ref_ax.text(0.5,1.15,'BASTA Reflectivity',c='dimgrey',fontweight='bold',fontsize=Fontsize*1.75,transform=date1_basta_ref_ax.transAxes,ha='center',va='center')
date2_basta_ref_ax.text(0.5,1.15,'BASTA Reflectivity',c='dimgrey',fontweight='bold',fontsize=Fontsize*1.75,transform=date2_basta_ref_ax.transAxes,ha='center',va='center')
date1_basta_vel_ax.text(0.5,1.15,'BASTA Doppler Velocity',c='dimgrey',fontweight='bold',fontsize=Fontsize*1.75,transform=date1_basta_vel_ax.transAxes,ha='center',va='center')
date2_basta_vel_ax.text(0.5,1.15,'BASTA Doppler Velocity',c='dimgrey',fontweight='bold',fontsize=Fontsize*1.75,transform=date2_basta_vel_ax.transAxes,ha='center',va='center')
date1_ceil_ax.text(0.5,1.15,'Univ. of Canterbury Ceilometer $\\beta_{att}$',c='dimgrey',\
                   fontweight='bold',fontsize=Fontsize*1.67,transform=date1_ceil_ax.transAxes,ha='center',va='center')
date2_ceil_ax.text(0.5,1.15,'Univ. of Canterbury Ceilometer $\\beta_{att}$',c='dimgrey',\
                   fontweight='bold',fontsize=Fontsize*1.75,transform=date2_ceil_ax.transAxes,ha='center',va='center')
date1_sfc_ax.text(0.5,1.1,'Surface Meteorology',c='dimgrey',\
                   fontweight='bold',fontsize=Fontsize*1.67,transform=date1_sfc_ax.transAxes,ha='center',va='center')
date2_sfc_ax.text(0.5,1.1,'Surface Meteorology',c='dimgrey',\
                   fontweight='bold',fontsize=Fontsize*1.75,transform=date2_sfc_ax.transAxes,ha='center',va='center')

# Date 1 Sonde 1
dumstr=date1_sonde1_time.strftime('%H%M')
dumstr2=date1_sonde1_time.strftime('%m/%d/%Y')
dumstr='Sounding 1 @ '+dumstr+' UTC\non '+dumstr2
date1_sonde1_ax.text(0.5,1.3,dumstr,c='dimgrey',fontweight='bold',fontsize=Fontsize*1.5,transform=date1_sonde1_ax.transAxes,ha='center',va='center')

# Date 1 Sonde 2
dumstr=date1_sonde2_time.strftime('%H%M')
dumstr2=date1_sonde2_time.strftime('%m/%d/%Y')
dumstr='Sounding 2 @ '+dumstr+' UTC\non '+dumstr2
date1_sonde2_ax.text(0.5,1.3,dumstr,c='dimgrey',fontweight='bold',fontsize=Fontsize*1.5,transform=date1_sonde2_ax.transAxes,ha='center',va='center')

# Date 2 Sonde 1
dumstr=date2_sonde1_time.strftime('%H%M')
dumstr2=date2_sonde1_time.strftime('%m/%d/%Y')
dumstr='Sounding 1 @ '+dumstr+' UTC\non '+dumstr2
date2_sonde1_ax.text(0.5,1.3,dumstr,c='dimgrey',fontweight='bold',fontsize=Fontsize*1.5,transform=date2_sonde1_ax.transAxes,ha='center',va='center')

# Date 2 Sonde 2
dumstr=date2_sonde2_time.strftime('%H%M')
dumstr2=date2_sonde2_time.strftime('%m/%d/%Y')
dumstr='Sounding 2 @ '+dumstr+' UTC\non '+dumstr2
date2_sonde2_ax.text(0.5,1.3,dumstr,c='dimgrey',fontweight='bold',fontsize=Fontsize*1.5,transform=date2_sonde2_ax.transAxes,ha='center',va='center')

#======================================
# Misc. Fixes
#======================================
date1_sonde1_ax.set_xlim(5,8)
date1_sonde2_ax.set_xlim(5,8)
date2_sonde1_ax.set_xlim(1.75,7)
date2_sonde2_ax.set_xlim(1.75,7)


#======================================
# Plot Sonde Paths
#======================================
import matplotlib.patheffects as pe

date1_basta_ref_ax.plot(date1_sonde1_time_long,date1_sonde1_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])
date1_basta_ref_ax.plot(date1_sonde2_time_long,date1_sonde2_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])
date2_basta_ref_ax.plot(date2_sonde1_time_long,date2_sonde1_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])
date2_basta_ref_ax.plot(date2_sonde2_time_long,date2_sonde2_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])

date1_basta_vel_ax.plot(date1_sonde1_time_long,date1_sonde1_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])
date1_basta_vel_ax.plot(date1_sonde2_time_long,date1_sonde2_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])
date2_basta_vel_ax.plot(date2_sonde1_time_long,date2_sonde1_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])
date2_basta_vel_ax.plot(date2_sonde2_time_long,date2_sonde2_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])

date1_ceil_ax.plot(date1_sonde1_time_long,date1_sonde1_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])
date1_ceil_ax.plot(date1_sonde2_time_long,date1_sonde2_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])
date2_ceil_ax.plot(date2_sonde1_time_long,date2_sonde1_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])
date2_ceil_ax.plot(date2_sonde2_time_long,date2_sonde2_height*1.e-3,c='brown',lw=5,path_effects=[pe.Stroke(linewidth=10, foreground='deepskyblue'), pe.Normal()])

# Locate Soundings
fac = 1.1
# Date 1
# Sonde1
date1_basta_ref_ax.text(date1_sonde1_time_long[-1],(date1_sonde1_height[-1]*1.e-3)+0.05,'Sounding 1',c='brown',fontsize=Fontsize*fac,fontweight='bold',ha='left',va='center')
date1_basta_vel_ax.text(date1_sonde1_time_long[-1],(date1_sonde1_height[-1]*1.e-3)+0.05,'Sounding 1',c='brown',fontsize=Fontsize*fac,fontweight='bold',ha='left',va='center')
date1_ceil_ax.text(date1_sonde1_time_long[-1],(date1_sonde1_height[-1]*1.e-3)+0.05,'Sounding 1',c='brown',fontsize=Fontsize*fac,fontweight='bold',ha='left',va='center')
# Sonde2
date1_basta_ref_ax.text(date1_sonde2_time_long[-1],(date1_sonde2_height[-1]*1.e-3)+0.05,'Sounding 2',c='brown',fontsize=Fontsize*fac,fontweight='bold',ha='left',va='center')
date1_basta_vel_ax.text(date1_sonde2_time_long[-1],(date1_sonde2_height[-1]*1.e-3)+0.05,'Sounding 2',c='brown',fontsize=Fontsize*fac,fontweight='bold',ha='left',va='center')
date1_ceil_ax.text(date1_sonde2_time_long[-1],(date1_sonde2_height[-1]*1.e-3)+0.05,'Sounding 2',c='brown',fontsize=Fontsize*fac,fontweight='bold',ha='left',va='center')

# Date 2
# Sonde1
date2_basta_ref_ax.text(date2_sonde1_time_long[-1],(date2_sonde1_height[-1]*1.e-3)+0.05,'Sounding 1',c='brown',fontsize=Fontsize*fac,fontweight='bold',ha='left',va='center')
date2_basta_vel_ax.text(date2_sonde1_time_long[-1],(date2_sonde1_height[-1]*1.e-3)+0.05,'Sounding 1',c='brown',fontsize=Fontsize*fac,fontweight='bold',ha='left',va='center')
date2_ceil_ax.text(date2_sonde1_time_long[-1],(date2_sonde1_height[-1]*1.e-3)+0.05,'Sounding 1',c='brown',fontsize=Fontsize*fac,fontweight='bold',ha='left',va='center')
# Sonde2
date2_basta_ref_ax.text(date2_sonde2_time_long[-1],(date2_sonde2_height[-1]*1.e-3)+0.05,'Sounding 2',c='brown',fontsize=Fontsize*fac,fontweight='bold',ha='right',va='center')
date2_basta_vel_ax.text(date2_sonde2_time_long[-1],(date2_sonde2_height[-1]*1.e-3)+0.05,'Sounding 2',c='brown',fontsize=Fontsize*fac,fontweight='bold',ha='right',va='center')
date2_ceil_ax.text(date2_sonde2_time_long[-1],(date2_sonde2_height[-1]*1.e-3)+0.05,'Sounding 2',c='brown',fontsize=Fontsize*fac,fontweight='bold',ha='right',va='center')

plt.subplots_adjust(wspace=0.6,hspace=0.45)

xpos = 0.025
ypos = 0.875
fac = 2.25
date1_sonde1_ax.text(xpos,ypos,'(a)',fontsize=Fontsize*fac,transform=date1_sonde1_ax.transAxes,fontweight='bold')
date1_sonde2_ax.text(xpos,ypos,'(b)',fontsize=Fontsize*fac,transform=date1_sonde2_ax.transAxes,fontweight='bold')
date2_sonde1_ax.text(xpos,ypos,'(c)',fontsize=Fontsize*fac,transform=date2_sonde1_ax.transAxes,fontweight='bold')
date2_sonde2_ax.text(xpos,ypos,'(d)',fontsize=Fontsize*fac,transform=date2_sonde2_ax.transAxes,fontweight='bold')
date1_basta_ref_ax.text(xpos,ypos,'(e)',fontsize=Fontsize*fac,transform=date1_basta_ref_ax.transAxes,fontweight='bold')
date2_basta_ref_ax.text(xpos,ypos,'(f)',fontsize=Fontsize*fac,transform=date2_basta_ref_ax.transAxes,fontweight='bold')
date1_basta_vel_ax.text(xpos,ypos,'(g)',fontsize=Fontsize*fac,transform=date1_basta_vel_ax.transAxes,fontweight='bold')
date2_basta_vel_ax.text(xpos,ypos,'(h)',fontsize=Fontsize*fac,transform=date2_basta_vel_ax.transAxes,fontweight='bold')
date1_ceil_ax.text(xpos,ypos,'(i)',fontsize=Fontsize*fac,transform=date1_ceil_ax.transAxes,fontweight='bold')
date2_ceil_ax.text(xpos,ypos,'(j)',fontsize=Fontsize*fac,transform=date2_ceil_ax.transAxes,fontweight='bold')
date1_sfc_ax.text(xpos,ypos,'(k)',fontsize=Fontsize*fac,transform=date1_sfc_ax.transAxes,fontweight='bold')
date2_sfc_ax.text(xpos,ypos,'(l)',fontsize=Fontsize*fac,transform=date2_sfc_ax.transAxes,fontweight='bold')

fig_path = '/home/mwstanfo/figures/micre_paper/'
outfile = 'fig_a5.png'
plt.savefig(fig_path+outfile,dpi=200,bbox_inches='tight')
#plt.show()
plt.close()
print('done')



/home/mwstanfo/anaconda3/envs/act_env/lib/python3.7/site-packages/ipykernel_launcher.py:185: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.
/home/mwstanfo/anaconda3/envs/act_env/lib/python3.7/site-packages/ipykernel_launcher.py:349: UserWarning: The input coordinates to pcolormesh are interpreted as cell centers, but are not monotonically increasing or decreasing. This may lead to incorrectly calculated cell edges, in which case, please supply explicit cell edges to pcolormesh.


done


In [ ]:
def plot_sfc_met(sfc_met_ax1,sfc_met_ax2,sfc_met_dict,basta_dict):
    
    temperature = sfc_met_dict['native_temperature']
    pressure = sfc_met_dict['native_pressure']
    rh = sfc_met_dict['native_rh']
    time_dt = sfc_met_dict['native_time_dt']
    wind_speed = sfc_met_dict['native_wind_speed']
    wind_dir = sfc_met_dict['native_wind_dir']
    
    
    #--------------------------------------------
    # Temp, RH, Pressure
    #--------------------------------------------   
    sfc_met_ax1.plot(time_dt,temperature,lw=3,c='red')
    sfc_met_ax1.set_ylabel('Temperature [$^{\\circ}$C]',fontsize=Fontsize)
    sfc_met_ax1_rh = sfc_met_ax1.twinx()
    sfc_met_ax1_rh.set_ylabel('RH [%]',fontsize=Fontsize)
    sfc_met_ax1_rh.plot(time_dt,rh,lw=3,c='blue')

    # deal with axis colors
    sfc_met_ax1.spines['left'].set_color('red')
    sfc_met_ax1_rh.spines['right'].set_color('blue')
    sfc_met_ax1.yaxis.label.set_color('red')
    sfc_met_ax1_rh.yaxis.label.set_color('blue')
    sfc_met_ax1.tick_params(axis='y', colors='red')
    sfc_met_ax1_rh.tick_params(axis='y', labelsize=Fontsize, colors='blue')
    sfc_met_ax1_rh.spines['left'].set_visible(False)
    sfc_met_ax1_rh.set_ylim(0,100)
    
    # add pressure to plot
    sfc_met_ax1_pressure = sfc_met_ax1.twinx()
    sfc_met_ax1_pressure.plot(time_dt,pressure*10.,lw=3,c='black')
    sfc_met_ax1_pressure.set_ylabel('Surface Pressure [hPa]',fontsize=Fontsize)
    sfc_met_ax1_pressure.yaxis.set_label_position("right")
    sfc_met_ax1_pressure.yaxis.set_ticks_position("right")
    sfc_met_ax1_pressure.spines['right'].set_position(('axes', +1.1))
    sfc_met_ax1_pressure.yaxis.label.set_color('black')
    sfc_met_ax1_pressure.tick_params(axis='y',labelsize=Fontsize,colors='black')
    sfc_met_ax1_pressure.spines['right'].set_color('black')

    
    # Find contiguous regions where rh > 95.
    cloud_layer_mask = np.zeros(len(rh))
    dumid = np.where(rh >= 95.)
    if np.size(dumid) > 0.:
        cloud_layer_mask[dumid] = 1
    cloud_Objects,num_cloud_objects = ndimage.label(cloud_layer_mask)

    #dumy=sfc_met_ax1_rh.get_ylim()
    #dumy = np.arange(dumy[0],dumy[1]+1)

    for kk in range(num_cloud_objects):
        dumid = np.where(cloud_Objects == (kk+1))
        dumid = np.squeeze(dumid)
        if np.size(dumid) > 1:
            sfc_met_ax1_rh.axvspan(time_dt[dumid[0]],time_dt[dumid[-1]],facecolor='purple',alpha=0.25)



    #--------------------------------------------
    # Winds
    #-------------------------------------------- 
    sfc_met_ax2.plot(time_dt,wind_speed,lw=3,c='black')
    sfc_met_ax2.set_ylabel('Wind Speed [m s$^{-1}$]',fontsize=Fontsize)
    sfc_met_ax2_dir = sfc_met_ax2.twinx()
    sfc_met_ax2_dir.set_ylabel('Wind Direction [$^{\\circ}$]',fontsize=Fontsize)
    sfc_met_ax2_dir.scatter(time_dt,wind_dir,s=2,lw=3,c='blue')
    
    # deal with axis colors
    sfc_met_ax2_dir.spines['right'].set_color('blue')
    sfc_met_ax2_dir.yaxis.label.set_color('blue')
    sfc_met_ax2_dir.tick_params(axis='y', labelsize=Fontsize, colors='blue')
    sfc_met_ax2_dir.spines['left'].set_visible(False)
    
    sfc_met_ax1.set_axisbelow(False)
    sfc_met_ax2.set_axisbelow(False)
    

In [ ]:
# wrapper
Fontsize=20


dum_return = plot_figure(dates_lim,merged_dicts)
#future = client.submit(plot_figure,date,infile,y_lim)
#futures.append(future)
print(aaaa)


In [ ]:
#cluster = LocalCluster(n_workers=8,threads_per_worker=1,memory_limit='4GB',processes=True)
#cluster

In [ ]:
#client = Client(cluster)
#client